In [1]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType,StringType, FloatType
import pyspark
from pyspark.sql import SparkSession
import os

In [2]:
accessKeyId = "admin"
secretAccessKey = "password"

In [7]:
# spark = (
#     SparkSession.builder
#     .master('spark://spark-iceberg:7077')
#     .appName("Spark Nessie Iceberg Demo")
#     .config("spark.driver.memory", "16g")
#     .config("spark.hadoop.fs.s3a.aws.credentials.provider",
#                  "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")\
#     .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.5.2,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.96.1')
#     .config("spark.hadoop.fs.s3a.access.key", accessKeyId)
#     .config("spark.hadoop.fs.s3a.secret.key", secretAccessKey)
#     .config("spark.hadoop.fs.s3a.path.style.access", True)
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
#     .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
#     .config("spark.sql.catalog.nessie.ref", "main")
#     .config("spark.sql.catalog.nessie.authentication.type", "NONE")
#     .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
#     .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
#     .config("spark.sql.catalog.nessie.warehouse", "s3://warehouse/")
#     .config("fs.s3a.connection.ssl.enabled", "false")
#     .getOrCreate()
# )


In [3]:
spark = (
    SparkSession.builder.appName("Iceberg-Nessie")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.lakehouse_prod_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.lakehouse_prod_catalog.type", "nessie")
    .config("spark.sql.catalog.lakehouse_prod_catalog.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.lakehouse_prod_catalog.ref", "main")
    .config("spark.sql.catalog.lakehouse_prod_catalog.warehouse", "s3a://warehouse/")
    .config("spark.sql.catalog.lakehouse_prod_catalog.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.lakehouse_prod_catalog.s3.path-style-access", "true")
    .config("spark.sql.catalog.lakehouse_prod_catalog.s3.access-key-id", "admin")
    .config("spark.sql.catalog.lakehouse_prod_catalog.s3.secret-access-key", "password")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .getOrCreate()
)


25/10/15 15:53:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
+---------+



In [42]:
spark.sql("SHOW NAMESPACES").show()

+---------------+
|      namespace|
+---------------+
|        test_db|
|raw_bronze_zone|
+---------------+



In [44]:
spark.sql("SHOW CATALOGS").show()

+--------------------+
|             catalog|
+--------------------+
|lakehouse_prod_ca...|
|       spark_catalog|
+--------------------+



In [7]:
spark.sql("SHOW TABLES IN lakehouse_prod_catalog.raw_bronze_zone").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [26]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS lakehouse_prod_catalog.raw_bronze_zone;").show()

++
||
++
++



In [8]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS lakehouse_prod_catalog.test_db.sample_table (
            id BIGINT,
            name STRING
        ) USING iceberg;
    """
).show()

++
||
++
++



In [9]:
# Insert data into the sample_table
spark.sql(
    """
        INSERT INTO lakehouse_prod_catalog.test_db.sample_table VALUES
        (1, 'Alice'),
        (2, 'Bob');
    """
).show()

++
||
++
++



In [10]:
# Insert data into the sample_table
spark.sql(
    """
SHOW TABLES;
    """
).show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  test_db|sample_table|      false|
+---------+------------+-----------+



In [13]:
!pip install boto3


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [17]:
import boto3
import pandas as pd
from io import StringIO

# S3 Credential
aws_access_key_id = "admin"
aws_secret_access_key = "password"

# Create S3 Client for MinIO
s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    endpoint_url="http://minio:9000",  # MinIO endpoint
    use_ssl=False
)

# Bucket Name
bucket_name = "landing-data"

# List objects in bucket
response = s3.list_objects_v2(Bucket=bucket_name)
objects_list = response.get("Contents", [])

# Loop through objects and read CSV into Pandas DataFrame
dfs = []  # list to store DataFrames
for obj in objects_list:
    obj_name = obj["Key"]
    response = s3.get_object(Bucket=bucket_name, Key=obj_name)
    object_content = response["Body"].read().decode("utf-8")

    # Convert CSV text to Pandas DataFrame
    df = pd.read_csv(StringIO(object_content))
    dfs.append(df)

# Combine all files into a single DataFrame (optional)
final_df = pd.concat(dfs, ignore_index=True)

final_df.head()

,Unnamed: 0,FirstName,LastName,StartDate,ExitDate,Title,Supervisor,ADEmail,BusinessUnit,EmployeeStatus,...,Satisfaction Score,Work-Life Balance Score,Training Date,Training Program Name,Training Type,Training Outcome,Location,Trainer,Training Duration(Days),Training Cost
0,0,Uriah,Bridges,20-Sep-19,NaN,Production Technician I,Peter Oneill,uriah.bridges@bilearner.com,CCDR,Active,...,2,3,15-Jul-23,Leadership Development,Internal,Failed,South Marisa,Taylor Rodriguez,2,606.11
1,1,Paula,Small,11-Feb-23,NaN,Production Technician I,Renee Mccormick,paula.small@bilearner.com,EW,Active,...,1,5,12-Sep-22,Customer Service,External,Incomplete,Tammieville,Kelly Patterson DDS,4,673.02
2,2,Edward,Buck,10-Dec-18,NaN,Area Sales Manager,Crystal Walker,edward.buck@bilearner.com,PL,Active,...,2,1,13-Aug-22,Leadership Development,External,Failed,East Roberthaven,Taylor Thomas,2,413.28
3,3,Michael,Riordan,21-Jun-21,NaN,Area Sales Manager,Rebekah Wright,michael.riordan@bilearner.com,CCDR,Active,...,5,4,15-Dec-22,Project Management,External,Completed,Garzatown,Holly Elliott,3,663.78
4,4,Jasmine,Onque,29-Jun-19,NaN,Area Sales Manager,Jason Kim,jasmine.onque@bilearner.com,TNS,Active,...,5,3,13-Jul-23,Technical Skills,External,Failed,Lake Meganville,Donald Martinez,5,399.03


In [20]:
catalog = 'lakehouse_prod_catalog'
schema = 'raw_bronze_zone'
table_name = 'raw_bronze_hris_data'

df_spark = spark.createDataFrame(final_df)
try:
    df_spark.write.mode(mode).format("parquet").saveAsTable(f"{catalog}.{schema}.{table_name}")
    print(f"Successfully load data into {catalog}.{schema}.{table_name}")
except Exception as e:
    print(f"Failed to load data into {catalog}.{schema}.{table_name}: {e}")
    raise

Failed to load data into lakehouse_prod_catalog.raw_bronze_zone.raw_bronze_hris_data: name 'mode' is not defined


NameError: name 'mode' is not defined

In [31]:
# Query the data from the table
spark.sql("SELECT * FROM lakehouse_prod_catalog.raw_bronze_zone.bronze_hris_employee").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `lakehouse_prod_catalog`.`raw_bronze_zone`.`bronze_hris_employee` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [lakehouse_prod_catalog, raw_bronze_zone, bronze_hris_employee], [], false


In [23]:
# Extract a Data
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-04-01 00:00:18|  2021-04-01 00:21:54|            1.0|          8.4|       1.0|                 N|          79|         116|           1|       25.5|  3.0|    0.5|      5.8

In [24]:
df.writeTo("lakehouse_prod_catalog.test_db.test_table_001").createOrReplace()

In [29]:
read_df = spark.read.table("lakehouse_prod_catalog.test_db.test_table_001")
read_df.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-04-01 00:00:18|  2021-04-01 00:21:54|            1.0|          8.4|       1.0|                 N|          79|         116|           1|       25.5|  3.0|    0.5|      5.8

In [31]:
# df.write.mode("overwrite").format("parquet").saveAsTable("test_db.test_table_001")

In [25]:
# # Try with Nessie Catalog
# # df.write.mode("overwrite").format("iceberg").saveAsTable("nessie.test_db.test_table_002")
# df.writeTo("test_db.test_table_002").using("iceberg").createOrReplace()

In [8]:
# Show Catalog
spark.sql("SHOW CATALOGS").show()

+--------------------+
|             catalog|
+--------------------+
|lakehouse_prod_ca...|
|       spark_catalog|
+--------------------+



In [27]:
# spark.sql("SHOW NAMESPACES IN lakehouse_prod_catalog").show()

In [7]:
spark.stop()

In [5]:
spark